In [ ]:
!pip install -r requirements.txt

In [1]:
import os
from glob import glob
import numpy as np
from tqdm import tqdm
import random
import pandas as pd

#torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset,DataLoader

#mne
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf

#signal preprocess
import scipy.signal as ssig    

from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42

In [2]:
# seed 고정 함수 및 seed 고정
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed)

In [3]:
# train_path = '/home/maic-player/dataset/1_Train+Val'
# test_path = '/home/maic-player/dataset/2_Test'
train_path = '/home/maic-player/FINAL_SUBMISSION/npy_60000/train'
valid_path = '/home/maic-player/FINAL_SUBMISSION/npy_60000/valid'
test_path = '/home/maic-player/FINAL_SUBMISSION/npy_60000/test'
save_path = '/home/maic-player/FINAL_SUBMISSION/results'
save_name = 'deepsleepnet_60000_focal.pth'
csv_name = 'deepsleepnet_60000_focal.csv'

In [4]:
train_path_list = glob(os.path.join(train_path, '*.npy'))
valid_path_list = glob(os.path.join(valid_path, '*.npy'))
test_path_list = glob(os.path.join(test_path, '*.npy'))

In [5]:
class EDFDataLoader(Dataset):
    def __init__(self, mode):
        self.mode = mode

        if self.mode == 'train':
            self.dataset = train_path_list
            
        elif self.mode == 'valid':
            self.dataset = valid_path_list
                
        elif self.mode == 'test':
            self.dataset = test_path_list
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data_np = np.load(self.dataset[idx])
        if 'Normal' in self.dataset[idx]:
            label = 0
        else:
            label = 1

        if (self.mode == 'train') or (self.mode == 'valid'):
            return data_np, torch.tensor(label)
        elif self.mode == 'test':
            return data_np, self.dataset[idx]

In [6]:
train_dataset = EDFDataLoader('train')
valid_dataset = EDFDataLoader('valid')

train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=16)
valid_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=32)

In [7]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.5, bidirectional=True)

    def forward(self, x):
        # set initial hidden and cell states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).cuda() #RuntimeError: Input and hidden tensors are not at the same device
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).cuda()

        # forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        return out

class DeepSleepNet(nn.Module):

    def __init__(self, ch=24):
        super(DeepSleepNet, self).__init__()
        self.features_s = nn.Sequential(
            nn.Conv1d(ch, 64, 50, 6),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=8, stride=8),
            nn.Dropout(),
            nn.Conv1d(64, 128, 6),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, 128, 6),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, 128, 6),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        self.features_l = nn.Sequential(
            nn.Conv1d(ch, 64, 400, 50),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=4, stride=4),
            nn.Dropout(),
            nn.Conv1d(64, 128, 8),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, 128, 8),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, 128, 8),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        self.features_seq = nn.Sequential(
            BiLSTM(96640, 512, 2),
        )
        self.res = nn.Linear(96640, 1024)
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(1024, 2),
        )

    def forward(self, x):
        x_s = self.features_s(x)
        x_l = self.features_l(x)
        x_s = x_s.flatten(1,2)
        x_l = x_l.flatten(1,2)
        x = torch.cat((x_s, x_l),1) # [bs, 7296]
        x_seq = x.unsqueeze(1)
        x_blstm = self.features_seq(x_seq) # [bs, 1, 1024]
        x_blstm = torch.squeeze(x_blstm, 1)
        x_res = self.res(x)
        x = torch.mul(x_res, x_blstm)
        x = self.classifier(x)
        return x

In [8]:
# Reference: https://github.com/clcarwin/focal_loss_pytorch/blob/master/focalloss.py

class FocalLoss(nn.Module):
    def __init__(self, gamma=0.8, alpha=0.7, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [9]:
model = DeepSleepNet()
model.to(device)

# criterion = torch.nn.CrossEntropyLoss()
criterion = FocalLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100)

In [ ]:
# training
print('Start Training')
print('-'*30)

best_val_f1 = 0
# early_stop_cnt = 0
best_pred = []
for epoch in range(100):
    for idx, data in enumerate(tqdm(train_loader)):
        train_data, train_labels = data
        train_data, train_labels  = train_data.to(device).float(), train_labels.to(device)
        y_pred = model(train_data)
#         y_pred = torch.sigmoid(y_pred)
        loss = criterion(y_pred, train_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # validation  
    model.eval()
    with torch.no_grad():    
        test_pred = []
        test_true = [] 
        for jdx, data in enumerate(tqdm(valid_loader)):
            test_data, test_labels = data
            test_data = test_data.to(device).float()
            test_labels = test_labels.to(device)
            y_pred = model(test_data)
            y_pred = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            test_pred.append(y_pred.detach().cpu().numpy())
            test_true.append(test_labels.detach().cpu().numpy())

        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        val_f1 = f1_score(test_true, test_pred, average='macro')
        
        model.train()
        if best_val_f1 < val_f1:
            best_val_f1 = val_f1
#             early_stop_cnt = 0
            best_pred = test_pred
            torch.save(model.state_dict(), os.path.join(save_path, save_name))
#         else:
#             early_stop_cnt += 1
#             if early_stop_cnt == 5:
#                 print('Epoch=%s, BatchID=%s, Val_F1=%.4f, Best_Val_F1=%.4f'%(epoch, idx, val_f1, best_val_f1))
#                 sys.exit()

        print('Epoch=%s, BatchID=%s, Val_F1=%.4f, Best_Val_F1=%.4f'%(epoch, idx, val_f1, best_val_f1))
        print(classification_report(test_true, test_pred))
        
    scheduler.step()

Start Training
------------------------------


  0%|          | 0/15 [00:00<?, ?it/s]/tmp/ipykernel_228506/3491591181.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)
100%|██████████| 2/2 [00:07<00:00,  3.73s/it]


Epoch=0, BatchID=14, Val_F1=0.6528, Best_Val_F1=0.6528
              precision    recall  f1-score   support

           0       0.41      1.00      0.58        14
           1       1.00      0.57      0.72        46

    accuracy                           0.67        60
   macro avg       0.71      0.78      0.65        60
weighted avg       0.86      0.67      0.69        60



  0%|          | 0/15 [00:00<?, ?it/s]/tmp/ipykernel_228506/3491591181.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)
100%|██████████| 2/2 [00:07<00:00,  3.78s/it]
/miniconda/envs/vanila-maic2022/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/miniconda/envs/vanila-maic2022/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/miniconda/envs/vanila-maic2022/lib/python3.8/site-packa

Epoch=1, BatchID=14, Val_F1=0.4340, Best_Val_F1=0.6528
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.77      1.00      0.87        46

    accuracy                           0.77        60
   macro avg       0.38      0.50      0.43        60
weighted avg       0.59      0.77      0.67        60



...

  0%|          | 0/15 [00:00<?, ?it/s]/tmp/ipykernel_228506/3491591181.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)
100%|██████████| 2/2 [00:03<00:00,  1.98s/it]


Epoch=39, BatchID=14, Val_F1=0.6660, Best_Val_F1=0.7948
              precision    recall  f1-score   support

           0       0.62      0.36      0.45        14
           1       0.83      0.93      0.88        46

    accuracy                           0.80        60
   macro avg       0.73      0.65      0.67        60
weighted avg       0.78      0.80      0.78        60



  0%|          | 0/15 [00:00<?, ?it/s]/tmp/ipykernel_228506/3491591181.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)
100%|██████████| 2/2 [00:08<00:00,  4.28s/it]


Epoch=40, BatchID=14, Val_F1=0.6033, Best_Val_F1=0.7948
              precision    recall  f1-score   support

           0       0.37      0.93      0.53        14
           1       0.96      0.52      0.68        46

    accuracy                           0.62        60
   macro avg       0.67      0.73      0.60        60
weighted avg       0.82      0.62      0.64        60



  0%|          | 0/15 [00:00<?, ?it/s]/tmp/ipykernel_228506/3491591181.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)
100%|██████████| 15/15 [00:37<00:00,  2.26s/it]

In [ ]:
test_dataset = EDFDataLoader('test')
test_loader = DataLoader(dataset=test_dataset, shuffle=False, batch_size=16)

model = DeepSleepNet()
model.to(device)
model.load_state_dict(torch.load(os.path.join(save_path, save_name)))

In [ ]:
# testing
print('Start Testing')
print('-'*30)

model.eval()
with torch.no_grad():    
    test_pred = []
    data_path_lst = []
    for jdx, (test_data, data_path) in enumerate(test_loader):
        test_data = test_data.to(device).float()
        y_pred = model(test_data)
        y_pred = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        test_pred.append(y_pred.detach().cpu().numpy())
        data_path_lst.append(data_path)

    test_pred = np.concatenate(test_pred)
    data_path_lst = np.concatenate(data_path_lst)
    
print('Done')

In [ ]:
pred_info = {}

for pred, img_path in zip(test_pred, data_path_lst):
    file_name = img_path.split('/')[-1].replace('.edf', '')
    if pred == 1:
        pred_info[file_name] = 'OSA'
    else:
        pred_info[file_name] = 'Normal'
    

sort_pred = dict(sorted(pred_info.items()))
print(sort_pred.items())

submission = pd.DataFrame.from_dict([sort_pred]).T

submission.to_csv(os.path.join(save_path, csv_name), index=True, header=False)
submission.head()